In [1]:
datos=spark.read.csv("movielens_ratings.csv",header=True,
                    inferSchema=True)

In [2]:
datos.printSchema(),datos.show(2)

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
+-------+------+------+
only showing top 2 rows



(None, None)

In [3]:
datos.select("userId").distinct().count()

30

In [4]:
datos.select("movieId").distinct().count()

100

In [5]:
datos.count()

1501

In [6]:
datos.groupBy("userId").count()\
    .orderBy("count",ascending=False).show()

+------+-----+
|userId|count|
+------+-----+
|     6|   57|
|    14|   57|
|    22|   56|
|    11|   56|
|     4|   55|
|    12|   55|
|     7|   54|
|     9|   53|
|    23|   52|
|    24|   52|
|    18|   52|
|    28|   50|
|    26|   49|
|     8|   49|
|     1|   49|
|     5|   49|
|     0|   49|
|    19|   49|
|    21|   48|
|    13|   48|
+------+-----+
only showing top 20 rows



In [7]:
datos.where("userId=11 and movieId=12").show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|     12|   1.0|    11|
+-------+------+------+



In [8]:
a=input("Usuario: ")
b=input("Película: ")
res=datos.where("userId="+a+" and movieId="+b).collect()
if res:
    print("Valoración:",res[0]["rating"])
else:
    print("No existe usuario o película")


Usuario: 11
Película: 12
Valoración: 1.0


In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
als=ALS(userCol="userId",itemCol="movieId",ratingCol="rating")

In [11]:
entrena,evalua=datos.randomSplit([0.8,0.2])

In [12]:
modelo=als.fit(entrena)

In [13]:
pred=modelo.transform(evalua)

In [14]:
pred.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    13|  1.2940367|
|     31|   2.0|    25|   2.317525|
|     31|   1.0|    24|  1.8584934|
|     85|   2.0|    20|  1.9956895|
|     85|   1.0|     5|  1.6947737|
|     65|   1.0|    28| 0.80727684|
|     65|   1.0|    16|  0.3609742|
|     65|   2.0|    15|  1.4708514|
|     53|   1.0|     6|  2.5968661|
|     53|   1.0|    25|-0.12351179|
|     53|   5.0|    21|  2.8997996|
|     53|   3.0|    14|  2.6242466|
|     78|   1.0|    22|   1.241158|
|     78|   1.0|     1| 0.79901856|
|     78|   1.0|    20| 0.92646587|
|     78|   1.0|    24|  0.8610881|
|     34|   1.0|     0| 0.81533104|
|     81|   1.0|     1|  1.0292356|
|     81|   1.0|    16| 0.71667826|
|     81|   2.0|     5|  1.7917476|
+-------+------+------+-----------+
only showing top 20 rows



In [15]:
pred.describe("rating","prediction").show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|               292|               292|
|   mean|1.8595890410958904| 1.548519430860673|
| stddev|1.1858894767856605|0.7534991564776243|
|    min|               1.0|       -0.46960938|
|    max|               5.0|         3.9362803|
+-------+------------------+------------------+



In [16]:
from pyspark.ml.evaluation import RegressionEvaluator

In [17]:
evaluador=RegressionEvaluator(labelCol="rating")

In [18]:
evaluador.evaluate(pred)

1.0342004140287324

In [19]:
evaluador2=RegressionEvaluator(labelCol="rating",metricName="r2")

In [20]:
evaluador2.evaluate(pred)

0.23684835184306396

In [21]:
evaluador2.evaluate(pred.withColumn("prediction",pred.prediction+0.3))

0.3058034355530309

In [22]:
modeloOk=als.fit(datos)

In [23]:
modeloOk.recommendForAllItems(10).show(truncate=False)

+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movieId|recommendations                                                                                                                                                            |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|31     |[[12, 2.652423], [8, 2.5721335], [6, 2.4888673], [7, 2.4773223], [14, 2.3516285], [25, 2.2611465], [23, 2.082271], [21, 1.9511954], [9, 1.9420462], [16, 1.8493192]]       |
|85     |[[16, 4.089936], [8, 3.782813], [14, 3.7698863], [24, 3.47327], [7, 3.281516], [6, 3.1287875], [21, 2.7705276], [1, 2.109042], [22, 2.030056], [20, 2.0213199]]            |
|65     |[[23, 3.9449732], [12, 3.7246382], [11, 3.411], [9, 2.801858], [29, 2.741892], [1

In [24]:
modeloOk.recommendForAllUsers(3).show(truncate=False)

+------+---------------------------------------------------+
|userId|recommendations                                    |
+------+---------------------------------------------------+
|28    |[[92, 4.487644], [81, 3.8866546], [2, 3.6120398]]  |
|26    |[[51, 4.9111547], [22, 4.6604505], [88, 4.45104]]  |
|27    |[[32, 3.160455], [18, 3.1571398], [23, 3.000766]]  |
|12    |[[46, 5.599124], [55, 4.51141], [17, 4.0952916]]   |
|22    |[[75, 4.5750003], [88, 4.3966336], [74, 4.3653593]]|
|1     |[[68, 2.9469347], [62, 2.868292], [22, 2.6379557]] |
|13    |[[93, 2.9901652], [74, 2.7195876], [53, 2.6859486]]|
|6     |[[25, 3.7054513], [29, 3.2215073], [85, 3.1287875]]|
|16    |[[90, 4.3030324], [85, 4.089936], [51, 3.7520874]] |
|3     |[[51, 4.114564], [75, 3.9761112], [30, 3.700545]]  |
|20    |[[22, 3.7188187], [94, 3.2269902], [68, 3.0894015]]|
|5     |[[46, 3.7487206], [55, 3.619256], [90, 3.6158292]] |
|19    |[[22, 3.389442], [94, 3.3387182], [90, 3.1782134]] |
|15    |[[46, 3.8619726]

In [33]:
nuevo=spark.createDataFrame([(12,0.0,110)],datos.schema)

In [34]:
modeloOk.transform(nuevo).show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     12|   0.0|   110|       NaN|
+-------+------+------+----------+

